# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
# Step 1: Read and preprocess the movie_review_*.csv files

# Note: All the dataframes have different column names.
# Column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng['Original Language'] = "English"
    df_eng = df_eng.rename(columns={'Movie / TV Series':'Title'})
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr['Original Language'] = "French"
    df_fr = df_fr.rename(columns={'Titre':'Title','Année':'Year', 'Critiques':'Review'})
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp['Original Language'] = "Spanish"
    df_sp = df_sp.rename(columns={'Título':'Title','Año':'Year', 'Sinopsis':'Synopsis', 'Críticas':'Review'})
    
    df = pd.concat([df_eng,df_fr,df_sp])
    
    return df

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
3,Les Choristes,2004,Ce film raconte l'histoire d'un professeur de ...,"""Les Choristes est un film magnifique qui vous...",French
7,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",Spanish
5,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",Spanish
8,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",Spanish
7,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",French
1,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",Spanish
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
2,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",Spanish
0,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",Spanish
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# Step 2: Load translation models and tokenizers
from transformers import MarianMTModel, MarianTokenizer

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# Step 3: Create the translate function
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
# Step 4: Translate the reviews and synopsis

# filter reviews in French and translate to English
fr_reviews = df.Review[df["Original Language"]=="French"]
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df.Synopsis[df["Original Language"]=="French"]
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df.Review[df["Original Language"]=="Spanish"]
es_reviews_en = es_reviews.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df.Synopsis[df["Original Language"]=="Spanish"]
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [6]:
# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df2 = df.copy()
df2.loc[df2['Original Language']=='French', 'Review'] = fr_reviews_en
df2.loc[df2['Original Language']=='Spanish', 'Review'] = es_reviews_en
df2.loc[df2['Original Language']=='French', 'Synopsis'] = fr_reviews_en
df2.loc[df2['Original Language']=='Spanish', 'Synopsis'] = es_reviews_en
df2.sample(5)

,Title,Year,Synopsis,Review,Original Language
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
9,Babylon A.D.,2008,"""This film is a complete mess. The characters ...","""This film is a complete mess. The characters ...",French
0,Roma,2018,"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish
8,Torrente: El brazo tonto de la ley,1998,"""Torrente is a vulgar and offensive film that ...","""Torrente is a vulgar and offensive film that ...",Spanish
3,El Laberinto del Fauno,2006,"""The Labyrinth of Fauno is a fascinating and e...","""The Labyrinth of Fauno is a fascinating and e...",Spanish


In [7]:
df = df2.copy()

In [9]:
df.to_csv("df_post_translation.csv")

In [2]:
df = pd.read_csv("df_post_translation.csv")

### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [3]:
# Step 5.1: Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

df.Review[0]

'"The Shawshank Redemption is an inspiring tale of hope and friendship, with outstanding performances by Tim Robbins and Morgan Freeman. The story is incredibly moving and will stay with you long after the credits roll."'

In [13]:
# Step 5.2: Load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

sentiment_classifier = pipeline('text-classification', model=model_name)#, tokenizer=tokenizer)

In [14]:
# Step 6: Creating the sentiment function
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)
    label = result[0]["label"]
    score = result[0]["score"]
    return label, score

In [19]:
# Step 7: Conducting sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df["sentiment_label"] = df["Review"].apply(lambda x : analyze_sentiment(x, sentiment_classifier)[0])
df["sentiment_score"] = df["Review"].apply(lambda x : analyze_sentiment(x, sentiment_classifier)[1])

In [20]:
df.sample(10)

,Unnamed: 0,Title,Year,Synopsis,Review,Original Language,sentiment_label,sentiment_score
18,8,Les Visiteurs en Amérique,2000,"""The film is a total waste of time. The jokes ...","""The film is a total waste of time. The jokes ...",French,NEGATIVE,0.999819
20,0,Roma,2018,"""Rome is a beautiful and moving film that pays...","""Rome is a beautiful and moving film that pays...",Spanish,POSITIVE,0.999880
12,2,Amélie,2001,"""Amélie is an absolutely charming film that wi...","""Amélie is an absolutely charming film that wi...",French,POSITIVE,0.999892
7,7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English,NEGATIVE,0.999802
16,6,La Tour Montparnasse Infernale,2001,"""I can't believe I've wasted time watching thi...","""I can't believe I've wasted time watching thi...",French,NEGATIVE,0.999815
4,4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English,POSITIVE,0.999885
25,5,Águila Roja,(2009-2016),"""Red Eagle is a boring and uninteresting serie...","""Red Eagle is a boring and uninteresting serie...",Spanish,NEGATIVE,0.999811
22,2,Y tu mamá también,2001,"""And your mom is also a movie that stays with ...","""And your mom is also a movie that stays with ...",Spanish,POSITIVE,0.999874
8,8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English,NEGATIVE,0.999799
13,3,Les Choristes,2004,"""The Choristes are a beautiful film that will ...","""The Choristes are a beautiful film that will ...",French,POSITIVE,0.999893


In [21]:
# export the results to a .csv file
df.to_csv("reviews_with_sentiment.csv", index=False)

In some cases a further differentiation, e.g. for overall statistical calculations, a model with a **star** rating: 1 - 5 is preferred, see [this](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment?text=I+like+you.) Bert-based model in the link.

In total, overall these are the most popular sentiment classification models on HuggingFace:

- **BERT** (Bidirectional Encoder Representations from Transformers): BERT is a transformer-based model that uses a bidirectional approach to capture the context of words in a sentence. It has achieved state-of-the-art performance on various natural language processing tasks, including sentiment classification.

- **RoBERTa** (Robustly Optimized BERT Pretraining Approach): RoBERTa is an optimized version of BERT that uses larger batch sizes and more training data. It has shown improved performance on several benchmarks, including sentiment analysis.

- **DistilBERT DistilBERT** is a smaller and faster version of BERT that retains most of its performance. It achieves this by using a knowledge distillation technique during training. It is suitable for resource-constrained environments.

- **XLNet**: XLNet is a generalized autoregressive pretraining method that outperforms BERT on various tasks. It considers all possible permutations of the input sequence to capture dependencies between all words, regardless of their positions.

- **GPT** (Generative Pretrained Transformer): GPT is a transformer-based language model that can be fine-tuned for sentiment classification. It uses a generative approach to predict the next word in a sequence, which helps capture the context and sentiment of the input text.